[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-1/chain.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/58238466-lesson-4-chain)

# Chain

## Review

We built a simple graph with nodes, normal edges, and conditional edges.

## Goals

Now, let's build up to a simple chain that combines 4 [concepts](https://python.langchain.com/v0.2/docs/concepts/):

* Using [chat messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as our graph state
* Using [chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) in graph nodes
* [Binding tools](https://python.langchain.com/v0.2/docs/concepts/#tools) to our chat model
* [Executing tool calls](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling) in graph nodes 

![Screenshot 2024-08-21 at 9.24.03 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dd607b08df5e1101_chain1.png)

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph

## Messages

Chat models can use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages), which capture different roles within a conversation. 

LangChain supports various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`. 

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call. 

Let's create a list of messages. 

Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, a message author 
* `response_metadata` - optionally, a dict of metadata (e.g., often populated by model provider for `AIMessages`)

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")]
messages.append(HumanMessage(content=f"Yes, that's right.",name="Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

## Chat Models

[Chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) can use a sequence of message as input and support message types, as discussed above.

There are [many](https://python.langchain.com/v0.2/docs/concepts/#chat-models) to choose from! Let's work with OpenAI. 

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

We can load a chat model and invoke it with out list of messages.

We can see that the result is an `AIMessage` with specific `response_metadata`.

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="glm-4.5-flash",
    temperature=0,
    api_key="e90a3144349c9571f2b465a17684099a.g1k1tPNphEaqpdbT",
    base_url="https://open.bigmodel.cn/api/paas/v4/",
)
result = llm.invoke(messages)
type(result)

In [ ]:
result

In [ ]:
result.response_metadata

In [ ]:
result.content

'That\'s an exciting question! Seeing orcas (killer whales) in the wild is a truly unforgettable experience. The "best" place depends on what you\'re looking for (guaranteed sightings vs. scenery, specific ecotypes, time of year), but here are the top locations in the US, ranked by reliability and popularity:\n\n**1. Pacific Northwest (San Juan Islands, Puget Sound, Strait of Juan de Fuca) - Washington State**\n\n*   **Why it\'s top:** This is the **undisputed best place to see the Southern Resident orcas**, a distinct and critically endangered population (only about 74 individuals remain). They are the iconic fish-eating orcas most people picture.\n*   **What you\'ll see:** Southern Resident pods (J, K, L pods). Occasionally, transient mammal-eating orcas (Bigg\'s) and offshore orcas pass through.\n*   **Best Time:** **May through September** is peak season. Residents are most reliably present from May to October, following salmon runs (especially Chinook). Sightings are never 100% guaranteed, but success rates are highest here for residents.\n*   **How:** Take a **dedicated whale watching tour** from Anacortes, Friday Harbor (San Juan Island), or Everett. Look for operators certified by the **Whale Watch Operators Association (WWOA)** and committed to responsible viewing practices (maintaining distance, following guidelines).\n*   **Pros:** Highest chance to see Southern Residents; stunning scenery; rich marine ecosystem; strong conservation focus.\n*   **Cons:** Sightings *can* still be unpredictable; very popular (book tours well in advance); requires planning around the season.\n\n**2. Alaska (Inside Passage - Juneau, Icy Strait Point, Frederick Sound, Stephens Passage) - Southeast Alaska**\n\n*   **Why it\'s top:** Offers incredible scenery and high chances to see **transient (Bigg\'s) orcas**, which hunt marine mammals. You might also see resident fish-eating orcas (distinct from the Southern Residents) and occasionally offshore orcas.\n*   **What you\'ll see:** Primarily transient orcas (Bigg\'s), which are more widespread and numerous than residents. Resident orcas (Northern Residents) are also present in some areas like Southeast Alaska. Humpbacks, humpbacks, sea lions, seals, porpoises, and bears are common bonuses.\n*   **Best Time:** **May through September**. Peak transient orca activity often coincides with the salmon runs and the presence of marine mammal prey (seals, sea lions, porpoises). Residents follow salmon runs too.\n*   **How:** Take whale watching tours from **Juneau, Icy Strait Point (Hoonah), or Petersburg**. Many cruise ships also offer good viewing opportunities in the Inside Passage.\n*   **Pros:** Stunning wilderness scenery; very high likelihood of seeing *some* type of orca (often transients); incredible biodiversity; less crowded than the PNW for orca-specific tours.\n*   **Cons:** Less focused on Southern Residents (you\'re more likely to see transients); weather can be more variable; requires traveling to Alaska.\n\n**3. California (Monterey Bay, Point Reyes, Farallon Islands) - California Coast**\n\n*   **Why it\'s top:** Offers year-round potential and a mix of ecotypes. Monterey Bay is particularly famous.\n*   **What you\'ll see:**\n    *   **Transient (Bigg\'s) Orcas:** Increasingly common, especially along the coast and around the Farallon Islands (hunting seals/sea lions).\n    *   **Offshore Orcas:** Less frequently seen, but known to inhabit deeper waters offshore.\n    *   **Resident Orcas:** Very rare, though a small pod (the "California Residents" or "Gulf of Alaska Residents") sometimes ventures south, especially in years of poor salmon abundance further north.\n*   **Best Time:** **Year-round**, with peaks:\n    *   **Transient Orcas:** Highest chance **late summer through fall** (Aug-Nov) when seal pups are abundant and prey is concentrated near the Farallones and coast.\n    *   **Offshore Orcas:** More likely **spring and summer**.\n    *   **Resident Orcas:** Highly unpredictable, but possible **late fall/winter** if salmon runs are poor further north.\n*   **How:** Whale watching tours from **Monterey Bay, Half Moon Bay, or Bodega Bay**. Trips to the **Farallon Islands** (longer, more expensive) offer the best transient viewing. Land-based viewing at **Point Reyes** can sometimes yield sightings.\n*   **Pros:** Year-round potential; high chance of seeing transients; world-class marine biodiversity; stunning coastline; proximity to major cities (SF, LA).\n*   **Cons:** Sightings are less predictable than PNW or Alaska; Southern Residents are extremely rare; weather can be foggy and rough; Farallon trips are expensive and long.\n\n**4. New England (Stellwagen Bank, Gulf of Maine) - Massachusetts**\n\n*   **Why it\'s top:** The primary location for the **North Atlantic Right Whale** (critically endangered), but also home to a small, distinct population of **offshore orcas**.\n*   **What you\'ll see:** **Offshore orcas** are the main draw here. They are less studied than residents or transients and feed on fish (like tuna). You\'ll also see humpbacks, fin whales, minke whales, dolphins, and seals.\n*   **Best Time:** **June through October**. Sightings are less frequent than in the West but are becoming more documented.\n*   **How:** Whale watching tours from **Boston, Gloucester, or Provincetown**.\n*   **Pros:** Unique East Coast orca population; excellent whale watching overall (humpbacks are a highlight); rich history and culture.\n*   **Cons:** Orcas are **not** a guaranteed sighting; much less predictable than the top West Coast spots; focused more on baleen whales.\n\n**Key Considerations for Choosing:**\n\n*   **Oca Type:** Do you specifically want to see the iconic Southern Residents (PNW), the widespread transients (Alaska, CA), or the less common offshores (CA, NE)?\n*   **Guarantee:** No wild animal sighting is ever 100% guaranteed. The PNW offers the *highest probability* for Southern Residents during peak season. Alaska offers high probability for *some* orcas (usually transients).\n*   **Time of Year:** Match your travel dates to the peak seasons above.\n*   **Scenery:** Alaska offers the most dramatic wilderness scenery. The PNW offers beautiful islands and waterways. California offers dramatic coastline.\n*   **Ethics:** **Choose responsible operators!** Look for those certified by organizations like WWOA (PNW) or adhering strictly to NOAA guidelines (maintaining 200 yards for Southern Residents, 400 yards for most other orcas). Avoid operators that chase or harass wildlife.\n*   **Other Wildlife:** All these locations offer incredible biodiversity beyond orcas (humpbacks, dolphins, seals, sea lions, birds, bears in AK).\n\n**Summary Table:**\n\n| Location          | Primary Orca Type(s)       | Best Time          | Key Advantage                                      | Key Consideration                                  |\n| :---------------- | :------------------------- | :----------------- | :------------------------------------------------- | :------------------------------------------------- |\n| **Pacific NW**    | Southern Residents (Endangered) | May - Sep          | **Highest chance for Southern Residents**          | Sightings not guaranteed; requires planning       |\n| **Alaska**        | Transient (Bigg\'s)         | May - Sep          | High chance of orcas; stunning wilderness scenery  | Less focused on Southern Residents; weather variable |\n| **California**    | Transient, Offshore        | Year-round (peaks) | Year-round potential; high transient sightings     | Less predictable; Southern Residents very rare     |\n| **New England**   | Offshore                   | Jun - Oct          | Unique East Coast population; great whale watching | Orcas not guaranteed; less frequent sightings      |\n\n**Recommendation:** For the most reliable and focused experience seeing the iconic Southern Resident orcas, **the Pacific Northwest (San Juan Islands/Puget Sound) during May-September is the top choice.** If you want stunning scenery and a high chance of seeing *any* orcas (likely transients), **Alaska is fantastic.** For year-round potential and transients, **California is excellent.** **New England** is a great option if you\'re already on the East Coast and want to see offshore orcas alongside other whales.\n\nEnjoy planning your orca adventure! It\'s an experience you\'ll never forget.'

## Tools

Tools are useful whenever you want a model to interact with external systems.

External systems (e.g., APIs) often require a particular input schema or payload, rather than natural language. 

When we bind an API, for example, as a tool we given the model awareness of the required input schema.

The model will choose to call a tool based upon the natural language input from the user. 

And, it will return an output that adheres to the tool's schema. 

[Many LLM providers support tool calling](https://python.langchain.com/v0.1/docs/integrations/chat/) and [tool calling interface](https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/) in LangChain is simple. 
 
You can simply pass any Python `function` into `ChatModel.bind_tools(function)`.

![Screenshot 2024-08-19 at 7.46.28 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbab08dc1c17a7a57f9960_chain2.png)

Let's showcase a simple example of tool calling!
 
The `multiply` function is our tool.

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])

If we pass an input - e.g., `"What is 2 multiplied by 3"` - we see a tool call returned. 

The tool call has specific arguments that match the input schema of our function along with the name of the function to call.

```
{'arguments': '{"a":2,"b":3}', 'name': 'multiply'}
```

In [ ]:
tool_call = llm_with_tools.invoke([HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")])
tool_call

In [ ]:
tool_call.tool_calls

## Using messages as state

With these foundations in place, we can now use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages) in our graph state.

Let's define our state, `MessagesState`, as a `TypedDict` with a single key: `messages`.

`messages` is simply a list of messages, as we defined above (e.g., `HumanMessage`, etc).

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

## Reducers

Now, we have a minor problem! 

As we discussed, each node will return a new value for our state key `messages`.

But, this new value [will override](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) the prior `messages` value.
 
As our graph runs, we want to **append** messages to our `messages` state key.
 
We can use [reducer functions](https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers) to address this.

Reducers allow us to specify how state updates are performed.

If no reducer function is specified, then it is assumed that updates to the key should *override it* as we saw before.
 
But, to append messages, we can use the pre-built `add_messages` reducer.

This ensures that any messages are appended to the existing list of messages.

We simply need to annotate our `messages` key with the `add_messages` reducer function as metadata.

In [ ]:
from typing import Annotated
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

Since having a list of messages in graph state is so common, LangGraph has a pre-built [`MessagesState`](https://langchain-ai.github.io/langgraph/concepts/low_level/#messagesstate)! 

`MessagesState` is defined: 

* With a pre-build single `messages` key
* This is a list of `AnyMessage` objects 
* It uses the `add_messages` reducer

We'll usually use `MessagesState` because it is less verbose than defining a custom `TypedDict`, as shown above.

In [ ]:
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built 
    pass

To go a bit deeper, we can see how the `add_messages` reducer works in isolation.

In [ ]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

## Our graph

Now, lets use `MessagesState` with a graph.

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
    
# Node
def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

If we pass in `Hello!`, the LLM responds without any tool calls.

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Hello!")})
for m in messages['messages']:
    m.pretty_print()

The LLM chooses to use a tool when it determines that the input or task requires the functionality provided by that tool.

In [ ]:
messages = graph.invoke({"messages": HumanMessage(content="Multiply 2 and 3")})
for m in messages['messages']:
    m.pretty_print()